In [33]:
from My_function_foody import *

In [55]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import random

In [60]:
links = read_file_txt("extracted_links.txt")

In [61]:
links = links[3117::]

In [58]:
def scroll_and_click_load_more(driver, pause_time=0.5, scroll_step=300, max_attempts=100):
    current_position = 0
    attempt = 0

    while attempt < max_attempts:
        driver.execute_script(f"window.scrollTo(0, {current_position});")
        time.sleep(pause_time)

        # Bấm tất cả "Xem thêm bình luận"
        while True:
            try:
                load_more_btn = driver.find_element(By.CSS_SELECTOR, "div.pn-loadmore[style*='display: block'] a.fd-btn-more")
                if load_more_btn.is_displayed():
                    driver.execute_script("arguments[0].scrollIntoView(true);", load_more_btn)
                    load_more_btn.click()
                    time.sleep(pause_time + 1)
                else:
                    break
            except:
                break

        # Kiểm tra hết nội dung chưa
        last_height = driver.execute_script("return document.body.scrollHeight")
        if current_position >= last_height:
            break

        current_position += scroll_step
        attempt += 1


In [62]:
chrome_options = Options()
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36")
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.set_page_load_timeout(180)


# --------------------- Bắt đầu lấy dữ liệu --------------------- #
base_url = "https://www.foody.vn"
output_file = "userdanhgias18.txt"
with open(output_file, "w", encoding="utf-8") as f_out:
    for link in links:
        full_link = base_url + link
        print(f"Đang xử lý link: {full_link}")

        # Mở trang với retry nếu lỗi
        retry_count = 0
        MAX_RETRY = 3
        while retry_count < MAX_RETRY:
            try:
                driver.get(full_link)
                print("Đã mở:", full_link)
                break
            except Exception as e:
                print(f"Lỗi mở link: {e}, thử lại lần {retry_count + 1}")
                retry_count += 1
                time.sleep(2)

        # Cuộn và bấm 'Xem thêm bình luận'
        scroll_and_click_load_more(driver)

        # Lấy dữ liệu HTML và phân tích với BeautifulSoup
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")
        review_items = soup.find_all("li", class_="review-item")

        # Lấy từng bình luận
        for item in review_items:
            username_tag = item.find("a", class_="ru-username")
            username = username_tag.text.strip() if username_tag else "Không rõ"
            profile_link = base_url + username_tag['href'] if username_tag and username_tag.has_attr('href') else ""

            avatar_tag = item.find("a", class_="ru-avatar").find("img")
            avatar_url = avatar_tag['src'] if avatar_tag else ""

            rating_tag = item.find("div", class_="review-points")
            rating = rating_tag.text.strip() if rating_tag else "Chưa có điểm"

            time_tag = item.find("span", class_="ru-time")
            time_posted = time_tag.text.strip() if time_tag else "Không rõ"

            device_tag = item.find("a", class_="ru-device")
            device = device_tag.text.strip() if device_tag else "Không rõ"

            content_tag = item.find("div", class_="rd-des").find("span", class_="ng-binding")
            content = content_tag.text.strip() if content_tag else "Không có nội dung"

            # Ghi vào file
            f_out.write(f"Tên người dùng: {username}\n")
            f_out.write(f"Link profile: {profile_link}\n")
            f_out.write(f"Avatar: {avatar_url}\n")
            f_out.write(f"Điểm đánh giá: {rating}\n")
            f_out.write(f"Thời gian: {time_posted}\n")
            f_out.write(f"Thiết bị: {device}\n")
            f_out.write(f"Nội dung bình luận: {content}\n")
            f_out.write(f"Link quán: {full_link}\n")
            f_out.write("="*80 + "\n")

        print(f"Đã thu thập {len(review_items)} bình luận từ: {full_link}\n")

Đang xử lý link: https://www.foody.vn/ha-noi/bao-bin-quan-nem-nuong-nha-trang
Đã mở: https://www.foody.vn/ha-noi/bao-bin-quan-nem-nuong-nha-trang
Đã thu thập 12 bình luận từ: https://www.foody.vn/ha-noi/bao-bin-quan-nem-nuong-nha-trang

Đang xử lý link: https://www.foody.vn/ha-noi/bun-dau-nhan
Đã mở: https://www.foody.vn/ha-noi/bun-dau-nhan
Đã thu thập 25 bình luận từ: https://www.foody.vn/ha-noi/bun-dau-nhan

Đang xử lý link: https://www.foody.vn/ha-noi/do-do-tra-sua-dong-gia-ho-tung-mau
Đã mở: https://www.foody.vn/ha-noi/do-do-tra-sua-dong-gia-ho-tung-mau
Đã thu thập 0 bình luận từ: https://www.foody.vn/ha-noi/do-do-tra-sua-dong-gia-ho-tung-mau

Đang xử lý link: https://www.foody.vn/ha-noi/gustoso-bistro-ha-hoi
Đã mở: https://www.foody.vn/ha-noi/gustoso-bistro-ha-hoi
Đã thu thập 24 bình luận từ: https://www.foody.vn/ha-noi/gustoso-bistro-ha-hoi

Đang xử lý link: https://www.foody.vn/ha-noi/nha-san-141
Đã mở: https://www.foody.vn/ha-noi/nha-san-141
Đã thu thập 45 bình luận từ: https:/